In [ ]:
import os
import shutil
import random

def split_class_data(dataset_dir, output_dir, test_size=0.2):
    """
    Splits each class's data into train and test subsets, while maintaining class structure.

    Args:
    - dataset_dir (str): Path to the original dataset containing class subdirectories.
    - output_dir (str): Path where the split dataset will be saved.
    - test_size (float): Fraction of data to be used for testing (default is 0.2).
    """
    # Define paths for train and test sets
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'validation')

    # Create train and test directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Iterate through each class folder in the dataset
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)

        # Skip non-directory files
        if not os.path.isdir(class_path):
            continue

        # Create corresponding class folders in train and test directories
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        # Get all files for the class and shuffle them
        files = os.listdir(class_path)
        random.shuffle(files)

        # Calculate split index
        split_index = int(len(files) * (1 - test_size))

        # Split files into train and test sets
        train_files = files[:split_index]
        test_files = files[split_index:]

        # Copy files to train directory
        for file in train_files:
            src = os.path.join(class_path, file)
            dst = os.path.join(train_dir, class_name, file)
            shutil.copy(src, dst)

        # Copy files to test directory
        for file in test_files:
            src = os.path.join(class_path, file)
            dst = os.path.join(test_dir, class_name, file)
            shutil.copy(src, dst)

        print(f"Processed class '{class_name}' - {len(train_files)} train, {len(test_files)} test")

    print(f"\nDataset split completed! Train data in '{train_dir}', Test data in '{test_dir}'.")

# Example usage
dataset_dir = "./dataset"  # Path to the original dataset with class subdirectories
output_dir = "./split_dataset"  # Path where the split dataset will be saved

# Split each class with 80% train, 20% test
split_class_data(dataset_dir, output_dir, test_size=0.2)


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Configure image size and directories
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Setup training and testing datasets (no augmentation)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    './split_dataset/train',  # Update with your actual train directory
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    './split_dataset/validation',  # Update with your actual validation directory
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False  # No shuffle for test set to ensure consistency during evaluation
)

# Load EfficientNetB0 base model with pre-trained weights
base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers initially

# Build the full model
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = base_model(inputs, training=False)  # Keep the base model in inference mode during training
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)  # GAP to reduce dimensionality
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)

model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Lower learning rate for fine-tuning
    metrics=["accuracy"]
)

# Train the model (Feature extraction phase)
initial_epochs = 5
history_all_classes_10_percent = model.fit(
    train_data_all_10_percent,
    epochs=initial_epochs,
    validation_data=test_data,
    validation_steps=int(0.15 * len(test_data))  # Validate on 15% of the data
)

# Unfreeze top 5 layers of the base model for fine-tuning
for layer in base_model.layers[:-5]:
    layer.trainable = False  # Keep earlier layers frozen

# Compile the model again after unfreezing some layers
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    metrics=["accuracy"]
)

# Fine-tune the model for additional epochs
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine_tune = model.fit(
    train_data_all_10_percent,
    epochs=total_epochs,
    validation_data=test_data,
    validation_steps=int(0.15 * len(test_data)),
    initial_epoch=history_all_classes_10_percent.epoch[-1]  # Start where the previous training stopped
)

# Evaluate the final model
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the fine-tuned model
model.save('efficientnet_fine_tuned_model.keras')
print("Model saved as 'efficientnet_fine_tuned_model.keras'")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the pre-trained model
model = tf.keras.models.load_model('/efficientnet_fine_tuned_model.keras')  # Replace with your model path

# Define the class names (order should match the training)
class_names = ['FreshApple', 'FreshBanana', 'FreshGrape', 'FreshGuava', 'FreshOrange', 'FreshPomegranate', 'FreshStrawberry', 'FreshJujube'
               'RottenApple', 'RottenBanana', 'RottenGrape', 'RottenGuava', 'RottenOrange', 'RottenPomegranate', 'RottenStrawberry', 'RottenJujube']  # Adjust based on your dataset labels

def predict_image_freshness(image_path):
    """
    Predicts if the provided image is fresh or rotten.

    Args:
    - image_path (str): Path to the input image.

    Returns:
    - Prediction result (str): Either 'fresh' or 'rotten'.
    """
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))  # Resize to match the input size of the model
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make a prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Display the result
    print(f"Prediction: {class_names[predicted_class]}")

    return class_names[predicted_class]

# Test the function with an image
image_path = '/WhatsApp Image 2024-10-20 at 23.32.39_d00905cc.jpg'  # Replace with the path to your test image
result = predict_image_freshness(image_path)
print(f'The provided image is: {result}')
